# Ejercicio 9

Utilizando los ejemplos del archivo AUTOS.csv genere un modelo utilizando un multiperceptrón para predecir el precio del auto (atributo price) y la cantidad de millas por galón en ruta (MPG-highway) en función del resto de los atributos.  

Recuerde completar los valores faltantes, utilizar normalización y dividir el dataset en entrenamiento y validación (80/20).  

Realice 20 ejecuciones independientes de cada configuración seleccionada calculando las épocas promedio y el error cuadrático medio (ECM).  

Analice los resultados y respalde las afirmaciones referidas a los resultados obtenidos.  
Utilice un máximo de 1000 épocas con lotes de 50 e implemente una parada temprana con paciencia de 15.


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DATA_PATH = "./Data/"

# ...existing code...
# leer marcando '?' como NA
data = pd.read_csv(DATA_PATH + 'autos.csv', na_values=['?'])

data = data.replace('?', np.nan)
# print(data.isnull().sum())
cols = ["normalized-losses", "bore", "stroke", "horsepower", "peak-rpm", "price"]

for col in cols:
    data[col] = pd.to_numeric(data[col], errors="coerce")
    data[col] = data[col].fillna(data[col].mean())

data = data.select_dtypes(include = ["int16", "int32", "int64", "float16", "float32", "float64"])

target_columns = [12, 14]  # mpg-highway y price
Y_raw = data.iloc[:, target_columns].values
X_raw = data.drop(data.columns[target_columns], axis=1).values

data_scaler, target_scaler = StandardScaler(), StandardScaler()

X_raw = data_scaler.fit_transform(X_raw)
Y_raw = target_scaler.fit_transform(Y_raw)



In [7]:
import tensorflow as tf

# Verifica si hay GPUs disponibles y configura para usar CUDA si es posible
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Configura TensorFlow para que use solo la memoria necesaria en la GPU
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    print("TensorFlow está configurado para usar CUDA (GPU).")
  except RuntimeError as e:
    print(e)
else:
  print("No se detectaron GPUs. TensorFlow usará la CPU.")


TensorFlow está configurado para usar CUDA (GPU).


# Inciso A

Complete la siguiente tabla y realice un análisis de los valores obtenidos:

**Tabla 1 — Épocas promedio**

| Optimizador | tanh | sigmoid | ReLU | LeakyReLU |
| ----------- | ---- | ------- | ---- | --------- |
| SGD         |      |         |      |           |
| RMSProp     |      |         |      |           |
| Adam        |      |         |      |           |

**Tabla 2 — ECM promedio**

| Optimizador | tanh | sigmoid | ReLU | LeakyReLU |
| ----------- | ---- | ------- | ---- | --------- |
| SGD         |      |         |      |           |
| RMSProp     |      |         |      |           |
| Adam        |      |         |      |           |



In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, r2_score
# import tensorflow as tf
import numpy as np

RUNS_PER_CONFIG = 20
MAX_EPOCHS = 1000
BATCH_SIZE = 50
TEST_SIZE = 0.2
PATIENCE = 15

ENTRADAS = X_raw.shape[1]
SALIDAS = Y_raw.shape[1]

# Separar datos
X_train, X_test, Y_train, Y_test = train_test_split(
    X_raw, Y_raw, test_size=TEST_SIZE, random_state=42
)

optimizers = ['sgd', 'rmsprop', 'adam']
activations = ['tanh', 'sigmoid', 'relu']

def get_optimizer(name):
    if name == 'sgd':
        return SGD()
    elif name == 'rmsprop':
        return RMSprop()
    elif name == 'adam':
        return Adam()

for opt in optimizers:
    for act in activations:
        epochs_list = []
        mse_list = []
        r2_list = []
        f1_list = []

        for run in range(RUNS_PER_CONFIG):
            # Definir modelo
            model = Sequential()
            model.add(Input(shape=(ENTRADAS,)))
            model.add(Dense(6, activation=act))
            model.add(Dense(3, activation=act))
            model.add(Dense(SALIDAS))

            model.compile(optimizer=get_optimizer(opt),
                          loss='mae', metrics=['mae', 'mse'])

            early_stop = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=PATIENCE, restore_best_weights=True
            )

            history = model.fit(
                x=X_train, y=Y_train,
                batch_size=BATCH_SIZE,
                epochs=MAX_EPOCHS,
                validation_data=(X_test, Y_test),
                callbacks=[early_stop],
                verbose=0
            )

            epochs_list.append(len(history.epoch))

            # Evaluar métricas de Keras
            loss, mae_val, mse_val = model.evaluate(X_test, Y_test, verbose=0)
            mse_list.append(mse_val)

            # Predicciones para métricas externas
            Y_pred = model.predict(X_test, verbose=0)

            # R2 score
            r2_list.append(r2_score(Y_test, Y_pred))

            # F1 score (si es clasificación binaria)
            # Y_test_bin = np.round(Y_test)
            # Y_pred_bin = np.round(Y_pred)
            # f1_list.append(f1_score(Y_test_bin, Y_pred_bin))

        mean_epochs = np.mean(epochs_list)
        mean_mse = np.mean(mse_list)
        mean_r2 = np.mean(r2_list)
        # mean_f1 = np.mean(f1_list)

        print(f"\nConfig -> Optimizador: {opt}, Activación: {act}")
        print(f" - Promedio de épocas: {mean_epochs:.2f}")
        print(f" - Promedio de MSE (validación): {mean_mse:.5f}")
        print(f" - Promedio R2 score: {mean_r2:.5f}")
        # print(f" - Promedio F1 score: {mean_f1:.5f}")



Config -> Optimizador: sgd, Activación: tanh
 - Promedio de épocas: 553.00
 - Promedio de MSE (validación): 0.34327
 - Promedio R2 score: 0.60711

Config -> Optimizador: sgd, Activación: sigmoid
 - Promedio de épocas: 946.20
 - Promedio de MSE (validación): 0.69019
 - Promedio R2 score: 0.26313

Config -> Optimizador: sgd, Activación: relu
 - Promedio de épocas: 513.95
 - Promedio de MSE (validación): 0.47382
 - Promedio R2 score: 0.48242

Config -> Optimizador: rmsprop, Activación: tanh
 - Promedio de épocas: 348.20
 - Promedio de MSE (validación): 0.33540
 - Promedio R2 score: 0.60670

Config -> Optimizador: rmsprop, Activación: sigmoid
 - Promedio de épocas: 819.70
 - Promedio de MSE (validación): 0.34679
 - Promedio R2 score: 0.58862

Config -> Optimizador: rmsprop, Activación: relu
 - Promedio de épocas: 374.10
 - Promedio de MSE (validación): 0.43187
 - Promedio R2 score: 0.52173

Config -> Optimizador: adam, Activación: tanh
 - Promedio de épocas: 374.60
 - Promedio de MSE (val

In [9]:
from tabnanny import verbose
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LeakyReLU
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, r2_score
# import tensorflow as tf
import numpy as np

RUNS_PER_CONFIG = 20
MAX_EPOCHS = 1000
BATCH_SIZE = 50
TEST_SIZE = 0.2
PATIENCE = 15

ENTRADAS = X_raw.shape[1]
SALIDAS = Y_raw.shape[1]

# Separar datos
X_train, X_test, Y_train, Y_test = train_test_split(
    X_raw, Y_raw, test_size=TEST_SIZE, random_state=42
)

optimizers = ['sgd', 'rmsprop', 'adam']
activation = 'leaky_relu'

def get_optimizer(name):
    if name == 'sgd':
        return SGD()
    elif name == 'rmsprop':
        return RMSprop()
    elif name == 'adam':
        return Adam()

for opt in optimizers:
    epochs_list = []
    mse_list = []
    r2_list = []
    f1_list = []

    for run in range(RUNS_PER_CONFIG):
        # Definir modelo con LeakyReLU
        model = Sequential()
        model.add(Input(shape=(ENTRADAS,)))
        model.add(Dense(6))
        model.add(LeakyReLU())
        model.add(Dense(3))
        model.add(LeakyReLU())
        model.add(Dense(SALIDAS))

        model.compile(optimizer=get_optimizer(opt),
                      loss='mae', metrics=['mae', 'mse'])

        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=PATIENCE, restore_best_weights=True
        )

        history = model.fit(
            x=X_train, y=Y_train,
            batch_size=BATCH_SIZE,
            epochs=MAX_EPOCHS,
            validation_data=(X_test, Y_test),
            callbacks=[early_stop],
            verbose=0
        )

        # Guardar número de épocas y MSE validación
        epochs_list.append(len(history.epoch))
        loss, mae_val, mse_val = model.evaluate(X_test, Y_test, verbose=0)
        mse_list.append(mse_val)

        # Predicciones
        Y_pred = model.predict(X_test, verbose=0)

        # R2 score (regresión)
        r2_list.append(r2_score(Y_test, Y_pred))

        # F1 score (si es clasificación binaria, convertir a 0/1)
        # Y_test_bin = np.round(Y_test)
        # Y_pred_bin = np.round(Y_pred)
        # f1_list.append(f1_score(Y_test_bin, Y_pred_bin))

    # Promedios
    mean_epochs = np.mean(epochs_list)
    mean_mse = np.mean(mse_list)
    mean_r2 = np.mean(r2_list)
    # mean_f1 = np.mean(f1_list)

    print(f"\nConfig -> Optimizador: {opt}, Activación: LeakyReLU")
    print(f" - Promedio de épocas: {mean_epochs:.2f}")
    print(f" - Promedio de MSE (validación): {mean_mse:.5f}")
    print(f" - Promedio R2 score: {mean_r2:.5f}")
    # print(f" - Promedio F1 score: {mean_f1:.5f}")



Config -> Optimizador: sgd, Activación: LeakyReLU
 - Promedio de épocas: 350.30
 - Promedio de MSE (validación): 0.30844
 - Promedio R2 score: 0.60200

Config -> Optimizador: rmsprop, Activación: LeakyReLU
 - Promedio de épocas: 298.85
 - Promedio de MSE (validación): 0.26678
 - Promedio R2 score: 0.63860

Config -> Optimizador: adam, Activación: LeakyReLU
 - Promedio de épocas: 287.40
 - Promedio de MSE (validación): 0.23358
 - Promedio R2 score: 0.68720
